## Set up API and some functions

In [1]:
!pip install openai==0.28

In [ ]:
import openai

# see: https://platform.openai.com/docs/quickstart
openai.api_key = ""  # add your key here

from IPython.display import display, Markdown
import requests
import json

In [3]:
# Define the DotDict class
class DotDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, dct):
        for key, value in dct.items():
            if hasattr(value, "keys"):
                value = DotDict(value)
            self[key] = value

In [13]:
def clinical_trials_api(condition, terms, intervention, topK=5):
    # Define the URL and make the request
    url = "https://clinicaltrials.gov/api/v2/studies"

    # Define the payload for the request
    payload = {
        "query.cond": condition,
        "query.term": terms,
        "query.intr": intervention,
        "sort": "@relevance:desc",
        # "filter.overallStatus": 'COMPLETED',
    }

    studies = []
    for request_num in range(0, topK, 10):
        response = requests.get(url, params=payload)
        data = response.json()
        studies.extend(data.get("studies", []))
        if "nextPageToken" in data:
            nextPageToken = data["nextPageToken"]
            payload["pageToken"] = nextPageToken
        else:
            break

    studies = studies[:topK]

    trial_ids = []
    clinical_trial_data = []

    # Check if the request was successful
    if response.status_code == 200:
        try:
            for idx, study_data in enumerate(studies):
                study = DotDict(study_data)
                try:
                    # Get the study number (nctId)
                    nct_id = study.protocolSection.identificationModule.nctId
                    trial_ids.append("https://clinicaltrials.gov/study/" + str(nct_id))
                    clinical_trial_data.append(
                        "Clinical Trial Details #{} in Nested JSON format:".format(
                            idx + 1
                        )
                    )
                    clinical_trial_data.append(json.dumps(study))
                except (KeyError, AttributeError):
                    continue

            # Save clinical trial data as a string in results_dict
            clinical_trial_data = "\n".join(clinical_trial_data)

        except requests.exceptions.JSONDecodeError:
            print("Failed to decode JSON. Here is the raw response:")
            print(response.text)
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)

    # Print the collected clinical trial data
    print("The following top {} clinical trials are pulled:".format(topK))
    print("\n".join(trial_ids))
    # print(clinical_trial_data)

    return trial_ids, clinical_trial_data

In [5]:
# Two options for a long input context LLM as of Nov 2024:
# 1) Use Claude 3.5 Haiku from Anthropic for 200k input context and 8192 tokens output,
# 2) Or use gpt-4o-mini from OpenAI for 128k input context and 16,384 tokens output


def chatgpt_summary(context):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # "o1-mini" (more expensive) or "gpt-4o-mini" (cheaper)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Please summarize the most clinically relevant information out of these nested JSON responses from a Clinical Trial Database for various related clinical trials. Interpret the JSON response values based on the key names which follow Camel Case. Please start with summary right away without any small talk. This is the trial data: {}".format(
                    context
                ),
            },
        ],
    )
    display(Markdown(response["choices"][0]["message"]["content"]))
    return response


def chatgpt_ask(context, question):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # "o1-mini" (more expensive) or "gpt-4o-mini" (cheaper)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Please thoroughly understand clinically important information encoded in the following nested JSON responses from a Clinical Trial Database for various related clinical trials. Interpret the JSON response values based on the key names which follow Camel Case. Please answer the subsequent questions based on this data. Please be factual and always cite the clinical trial source when you use a certain piece of information from there. Here is the trial data: {}".format(
                    context
                ),
            },
            {
                "role": "user",
                "content": "Now answer the following question: {}".format(question),
            },
        ],
    )
    display(Markdown(response["choices"][0]["message"]["content"]))
    return response

# Demo

In [6]:
condition = "infarction"
terms = "diabetes"
intervention = "heart attack"
top_number_of_trials_to_consider = 8

In [14]:
clinical_trials_context = clinical_trials_api(
    condition, terms, intervention, topK=top_number_of_trials_to_consider
)

The following top 8 clinical trials are pulled:
https://clinicaltrials.gov/study/NCT05431296
https://clinicaltrials.gov/study/NCT05709509
https://clinicaltrials.gov/study/NCT00191282
https://clinicaltrials.gov/study/NCT04564742
https://clinicaltrials.gov/study/NCT01882179
https://clinicaltrials.gov/study/NCT04509674
https://clinicaltrials.gov/study/NCT02880436
https://clinicaltrials.gov/study/NCT05645419


In [8]:
response = chatgpt_summary(clinical_trials_context)

### Summary of Clinical Trials

1. **NCT05431296 - Glucose Monitoring After Acute Myocardial Infarct in People With Diabetes (GLAM)**
   - **Status**: Recruiting
   - **Objective**: Evaluate the efficacy of continuous glucose monitoring (CGM) on glycemic control in type 2 diabetes patients post-acute myocardial infarction (AMI). 
   - **Design**: Randomized controlled trial (RCT) with 160 participants; cohorts include real-time CGM for 26 weeks and blinded CGM for 10 days.
   - **Primary Outcomes**: Percent time spent in glucose target range (3.9-10mmol/L) over 26 weeks.

2. **NCT05709509 - Effect of Colchicine on MMP-9, NOX2, and TGF-β1 in Myocardial Infarction**
   - **Status**: Recruiting
   - **Objective**: Investigate colchicine’s effect on biochemical markers after AMI.
   - **Design**: RCT with estimated enrollment of 148 participants (1:1 randomization between colchicine and standard therapy).
   - **Primary Outcomes**: Changes in serum levels of MMP-9, NOX2, and TGF-β1 from 24 hours to day 5 post-treatment.

3. **NCT00191282 - Hyperglycemia and Cardiovascular Outcomes With Type 2 Diabetes (HEART2D)**
   - **Status**: Completed
   - **Objective**: Compare two insulin strategies on cardiovascular outcomes in type 2 diabetic patients post-AMI.
   - **Design**: RCT with 1,116 participants.
   - **Primary Outcomes**: Incidence of primary combined cardiovascular events over 5.5 years.

4. **NCT04564742 - Dapagliflozin Effects on Cardiometabolic Outcomes in Patients With Acute Myocardial Infarction (DAPA-MI)**
   - **Status**: Completed
   - **Objective**: Determine the impact of dapagliflozin on heart failure and mortality in non-diabetic patients post-AMI.
   - **Design**: RCT with 4017 participants.
   - **Primary Outcomes**: Composite endpoint of hospitalizations for heart failure or all-cause mortality over 30 months.

5. **NCT01882179 - Mineralocorticoid Receptor Antagonist Treatment to Reduce Myocardial Infarct Size (MINIMISE-STEMI)**
   - **Status**: Completed
   - **Objective**: Investigate early MRA administration's effect on myocardial infarct size in STEMI patients.
   - **Design**: RCT with 61 participants.
   - **Primary Outcomes**: Myocardial infarct size at 12 weeks post-treatment.

6. **NCT04509674 - EMPACT-MI: Evaluate the Effect of Empagliflozin on Hospitalisation for Heart Failure and Mortality in Patients With Acute Myocardial Infarction**
   - **Status**: Completed
   - **Objective**: Evaluate empagliflozin for reducing heart failure hospitalizations and mortality after an AMI.
   - **Design**: RCT with 6522 participants.
   - **Primary Outcomes**: Time to first heart failure hospitalization or all-cause mortality.

7. **NCT02880436 - Association of Air Quality and Myocardial Infarction**
   - **Status**: Completed
   - **Objective**: Analyze the impact of air pollution on the occurrence of AMI.
   - **Design**: Observational study with 12,000 cases analyzed retrospectively.
   - **Primary Outcomes**: Correlation between air quality factors and daily myocardial infarction rates.

8. **NCT05645419 - Chronoprognosis of Myocardial Infarction in Patients With Type 2 Diabetes Mellitus**
   - **Status**: Not Yet Recruiting
   - **Objective**: Develop a mathematical model predicting MI based on biorhythms and external environmental factors.
   - **Design**: RCT with 60 participants; one group receiving personalized chronoprevention intervention.
   - **Primary Outcomes**: Incidence of myocardial infarction over 24 weeks.

### Key Findings
- **Support for Continuous Monitoring**: The trials emphasize the importance of innovation in diabetes management post-AMI, especially through the use of CGMs and pharmacological interventions targeting glycemic control.
- **Role of Inflammation Modulators**: Studies involving colchicine and mineralocorticoid receptor antagonists suggest potential benefits in modifying inflammation and cardiac remodeling post-infarction.
- **Impact of Cardiovascular Pharmacotherapy**: Dapagliflozin and empagliflozin's effects on cardiovascular outcomes could reshape post-AMI treatment strategies, highlighting the advances in diabetic care.
- **Environmental Considerations**: The link between air pollution and AMI incidence underscores a broader public health concern around environmental factors affecting heart health. 

These findings reflect ongoing research efforts to improve outcomes after myocardial infarction, particularly in diabetic populations, and explore innovative solutions in treatment practices.

In [9]:
question = """

What are the innovative ideas have been introduced in the clinical trials recently that are quite differentiating from the regular/conventional way
of executing clinical trials? Do the trials look hopeful at all? Are any of them still recruiting?

Also, answer the following questions please:
1) Which trial has more men registered than women?
2) Which trial deals with interventions that are generally thought to hold the most number of side affects than the others?
3) Can you also please return the SMILE strings of drugs used in each of the trials, if any.
4) If there is one thing you learnt from this list of trials that you haven't had an idea of before, what is that one learning?

"""
response = chatgpt_ask(clinical_trials_context, question.strip())

### Innovative Ideas in Recent Clinical Trials
1. **Personalized Chronoprevention**: The trial titled "Chronoprognosis of Myocardial Infarction in Patients With Type 2 Diabetes Mellitus" (NCT05645419) introduces the concept of using personalized chronobiofeedback from environmental factors, such as celestial rhythms, to tailor interventions for myocardial infarction. This innovative approach contrasts with traditional methods that focus primarily on pharmacologic interventions alone.
  
2. **Dapagliflozin in Non-Diabetic Patients**: The trial "Dapagliflozin Effects on Cardiometabolic Outcomes in Patients With an Acute Heart Attack" (NCT04564742) tests a drug typically used for diabetes management (SGLT2 inhibitors) in a population without diabetes. This trial seeks to evaluate its effects on heart failure and death, potentially broadening the use of SGLT2 inhibitors beyond conventional diabetes treatment.

3. **Colchicine for Myocardial Infarction**: The study examining "Effect of Colchicine on MMP-9, NOX2, and TGF-β1 in Myocardial Infarct" (NCT05709509) investigates an anti-inflammatory approach to post-myocardial infarction care. This study aims to redefine management strategies for myocardial infarction by lessening the inflammatory response.
  
### Hopefulness of the Trials
Many trials are showing promising exploratory outcomes, especially the studies involving continuous glucose monitoring (NCT05431296) and novel anti-inflammatory treatments like colchicine (NCT05709509). The concept of using potentially multifactorial approaches (like dapagliflozin) also suggests hopefulness for better cardiovascular outcomes.

### Current Recruitment Status
Among the trials:
- **Recruiting**: The trial "Glucose Monitoring After Acute Myocardial Infarct in People With Diabetes" (NCT05431296) is currently recruiting for 160 participants.
- **Completed**: Trials like "Hyperglycemia and Its Effect After Acute Myocardial Infarction on Cardiovascular Outcomes in Patients With Type 2 Diabetes (HEART2D)" (NCT00191282) have completed.

### Specific Questions
1. **Which trial has more men registered than women?**
   The trial "Hyperglycemia and Its Effect After Acute Myocardial Infarction on Cardiovascular Outcomes in Patients With Type 2 Diabetes (HEART2D)" (NCT00191282) has registered significantly more men (356) than women (201).

2. **Which trial deals with interventions thought to hold the most number of side effects?**
   The trial "Dapagliflozin Effects on Cardiometabolic Outcomes in Patients With an Acute Heart Attack" (NCT04564742) involves SGLT2 inhibitors, which are known to have a range of side effects including urinary tract infections and dehydration-related issues.

3. **SMILE Strings of Drugs Used in Each Trial**
   - **NCT05431296**: Device: Dexcom ONE Continuous Glucose Monitoring System
   - **NCT05709509**: Drug: Colchicine
   - **NCT00191282**: Drugs include Insulin lispro, Human insulin isophane suspension, Insulin glargine, Human insulin 30/70
   - **NCT04564742**: Drug: Dapagliflozin (Forxiga/ Farxiga)
   - **NCT01882179**: Drug: Spironolactone
   - **NCT04509674**: Drug: Empagliflozin
   - **NCT02880436**: No specific drug, observational study.

4. **One Learning from the Trials**
   One key learning from reviewing these trials is the increasing recognition of non-pharmacological factors, like chronobiology and personalized medicine, in the management and prevention of cardiovascular diseases. This interdisciplinary approach enhances the understanding of how both biological and environmental factors can influence patient outcomes, representing a shift from purely medication-focused treatment paradigms.